In [1]:
from sys import path
path.append('./src')
import pytest
from src.components.roster import Roster, ORC, TROLL
from src.components.game_map import MapCoords
from src.components.entities import BaseEntity


In [14]:
a = set([1, 2, 3])

In [16]:
a.add([4, 5])
a

TypeError: unhashable type: 'list'

In [ ]:
roster.live_ai_actors

In [ ]:
roster.get_entity_at_location(MapCoords(1, 1))

In [ ]:
from test.test_game_map import *
test_get_map_coords()
test_in_bounds()
test_is_traversable_transparent_visible_explored()
test_out_of_bounds_checks()
test_set_visible_and_reset()
test_update_explored()
test_width_height()
test_tiles_initialization()


In [2]:
from test.test_roster import *
test_roster_spawn()
test_roster_entity_blocked_locations()
test_roster_entity_collision()
test_roster_live_ai_actors()
test_roster_live_actors()
test_roster_get_entity_at_location()
